One important concept to understand when building chatbots is to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you passing in

## Setup and Important Libraries

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, trim_messages
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages.utils import count_tokens_approximately
from operator import itemgetter

In [2]:
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
llm = ChatGroq(model="Gemma2-9b-It")

## Prompt Template

In [5]:
system_message = "You are a helpful assistant. Answer all the questions to the best of your ability in {language}"

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="messages")
    ]
)

## Trim Messages

Helper to reduce how many messages we are sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [7]:
trimmer = trim_messages(
    max_tokens=100,
    strategy='last',
    token_counter=count_tokens_approximately,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [8]:
messages = [
    SystemMessage(content='You are a good assistant'),
    HumanMessage(content='Hi I am Saad'),
    AIMessage(content='Hi'),
    HumanMessage(content='I like to play video games'),
    AIMessage(content='That is great to know'),
    HumanMessage(content='What is 2 + 2'),
    AIMessage(content='The answer is 4'),
    HumanMessage(content='Thanks'),
    AIMessage(content='No Problems! Anything else?'),
    HumanMessage(content='No at the moment, See you next time'),
    AIMessage(content='Sure'),
]

In [9]:
trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like to play video games', additional_kwargs={}, response_metadata={}),
 AIMessage(content='That is great to know', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The answer is 4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No Problems! Anything else?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='No at the moment, See you next time', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Sure', additional_kwargs={}, response_metadata={})]

## Chain

In [10]:
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | llm
)

In [11]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What do i Like")],
        "language": "English"
    }
)

response.content

"As an AI, I don't have memory of past conversations.  \n\nTo know what you like, you'd have to tell me!  \n\nWhat are some things you enjoy? 😊  \n\n"

In [12]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English"
    }
)

response.content

'You asked "What is 2 + 2?"  😊 \n'

## Wrap in Message History

In [13]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
        
    return store[session_id]

In [14]:
config = {
    "configurable": {"session_id": "chat1"}
}

In [15]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [17]:
response = with_message_history.invoke(
    {
        'messages':[HumanMessage(content="What is my name?")],
        'language': 'English'
    },
    config=config
)

response.content

"As an AI, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n\n"